In [1]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm # to create loadbard in for loop
import os.path
import pickle

In [2]:
# need to precise the option for date, nbr of metada for call transcript in the url paramater 
# https://en.wikipedia.org/wiki/Query_string
nbr_earning_call_metada = 100
date_of_request= "2022-04-28"
url = "https://seekingalpha.com/api/v3/articles"+"?cacheBuster="\
      +str(date_of_request)\
      +"&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0"\
      +"&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]="\
      +str(nbr_earning_call_metada)\
      +"&page[number]=1"
url

'https://seekingalpha.com/api/v3/articles?cacheBuster=2022-04-28&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]=100&page[number]=1'

In [3]:
# the chrome driver depend on which version of chrome you are running specify the good one
executable_path = "./chromedriver_linux"
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path= executable_path,options=option)
driver.get(url)
site_html = driver.page_source

/home/thomas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [4]:
#a quick loock of the result to check if we get results are get captcha/bot error 
site_html[0:300]

'<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">{"data":[{"id":"4505020","type":"transcript","attributes":{"publishOn":"2022-04-29T07:53:10-04:00","isLockedPro":false,"commentCount":0,"gettyImageUrl":null,"themes":{},"title":"Brookline Bancorp Inc. (BRKL) CEO Paul'

In [5]:
api_json_response = BeautifulSoup(site_html)
api_data_response = json.loads(api_json_response.get_text())

## API data response

<pre>
dict( 
    data : list(one element is one earning call metada)
    included : "don't care for now"
    meta : "don't care for now"
        )
<pre>
One earning call metadata is show below 

In [6]:
api_data_response["data"][0]

{'id': '4505020',
 'type': 'transcript',
 'attributes': {'publishOn': '2022-04-29T07:53:10-04:00',
  'isLockedPro': False,
  'commentCount': 0,
  'gettyImageUrl': None,
  'themes': {},
  'title': 'Brookline Bancorp Inc. (BRKL) CEO Paul Perrault on Q1 2022 Results - Earnings Call Transcript'},
 'relationships': {'author': {'data': {'id': '44211', 'type': 'author'}},
  'sentiments': {'data': []},
  'primaryTickers': {'data': [{'id': '3497', 'type': 'tag'}]},
  'secondaryTickers': {'data': []},
  'otherTags': {'data': [{'id': '96991', 'type': 'tag'},
    {'id': '17862', 'type': 'tag'},
    {'id': '49', 'type': 'tag'},
    {'id': '326', 'type': 'tag'},
    {'id': '17894', 'type': 'tag'}]}},
 'links': {'self': '/article/4505020-brookline-bancorp-inc-brkl-ceo-paul-perrault-on-q1-2022-results-earnings-call-transcript'}}

In [7]:

path_to_earning_calls = [ earning_call_metada["links"]["self"] 
                     for earning_call_metada in api_data_response["data"]]
# ""check url
path_to_earning_calls[0:2]

['/article/4505020-brookline-bancorp-inc-brkl-ceo-paul-perrault-on-q1-2022-results-earnings-call-transcript',
 '/article/4504995-basf-se-basfy-ceo-martin-brudermuller-on-q1-2022-results-earnings-call-transcript']

## download all not yet download html from url

In [8]:
domain_name = 'https://seekingalpha.com'
need_to_download = []
for path_to_earning_call in tqdm(path_to_earning_calls):
    if not os.path.isfile("data"+ path_to_earning_call+".html"):
        # not in local directory
        need_to_download.append(path_to_earning_call)
print(f"Need to download {len(need_to_download)} html file(s)")
for path_to_earning_call in tqdm(need_to_download):
    with open("data"+ path_to_earning_call+".html", "xb") as file:
            url = domain_name + path_to_earning_call
            html_transcript = requests.get(url).text
            pickle.dump(html_transcript, file)
    

100%|████████████████████████████████████████| 50/50 [00:00<00:00, 16024.70it/s]


Need to download 46 html file(s)


100%|███████████████████████████████████████████| 46/46 [00:30<00:00,  1.49it/s]


## Check last file got downloaded, i.e. captcha/bot error 

In [16]:
BeautifulSoup(html_transcript).get_text()[:2000]

"McGrath RentCorp's (MGRC) CEO Joe Hanna on Q1 2022 Results - Earnings Call Transcript | Seeking Alpha                    Seeking Alpha - Go to HomepageTrendingMy PortfolioMy AuthorsTop StocksLatest NewsMarketsStock IdeasDividendsETFsEducationWatch & ListenTranscriptsServicesMcGrath RentCorp's (MGRC) CEO Joe Hanna on Q1 2022 Results - Earnings Call TranscriptApr. 28, 2022 9:13 PM ETMcGrath RentCorp (MGRC)SA Transcripts126.02K FollowersFollowMcGrath RentCorp (NASDAQ:MGRC) Q1 2022 Earnings Conference Call April 28, 2022 5:00 PM ET Company Participants Joe Hanna - Chief Executive Officer Keith Pratt - Chief Financial Officer Conference Call Participants Scott Schneeberger - Oppenheimer Operator Ladies and gentlemen, thank you for standing by. Welcome to the McGrath RentCorp First Quarter 2022 Earnings Conference call. At this time, all conference participants are in a listen-only mode. Later we will conduct a question-and-answer session. [Operator Instructions] This conference is being re